# Business questions with SQL

The situation is the following: we have a database from a music store. The database has the following structure, in which each table is connected via an id:

<img src="schema.png" width="500">

We have to answer two questions based on the data.

### Problem 1

Our first problem is the following: the store received a proposal to acquire albums four artists, each one belonging to some different genre. The genres were: `Hip-Hop`, `Punk`, `Pop` and `Blus`.
Now, the problem is the store can only choose between three artists, so we have to see which genres are the best for the store to acquire -- with the knowledge the store will sell these to the USA only.



In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Our strategy to solve the problem here is very straightforward: make a table containing the genres, how many sales it produced and the percentage of the total sales it represents.

Since we're interested in sales in the USA only, first we create a new table `count_usa` that counts the total of sales in the US (we do this to make the code cleaner). Then we simply unite the relevant info from the various tables.

In [2]:
%%sql

WITH count_usa AS (
    SELECT
        COUNT(*)
    FROM invoice_line il
    INNER JOIN invoice i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = 'USA'
)

SELECT
    g.name AS Genre,
    COUNT(il.quantity) AS Sold,
    ROUND((CAST(COUNT(il.quantity) AS Float)/(SELECT * FROM count_usa))*100, 2) AS Percentage
FROM track t
INNER JOIN genre g ON g.genre_id = t.genre_id
INNER JOIN invoice_line il ON il.track_id = t.track_id
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
WHERE i.billing_country = 'USA'
GROUP BY 1
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


Genre,Sold,Percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


We see from the table that the store should not purchase the `Hip-Hop` artist, since the genre had a poor performance in sales compared to the other three.

### Problem 2

The store wants to invest in some other countries and wants to know which one has the most potential.

So, to answer this question we come up with the following table: one column with the country and others with the total clients the store has in it; the total profit it generated; the avarage of profit and the avarage profit per order.

The caveat here is: a bunch of countries only have one client. The store is not interested in those countries.

Our strategy is, then: we create a new table `customer_grouped_one` that makes three things: 1. it groups all countries with one client together in the name `Other`; 2. it creates a column names `sort` that is `1` if the country is `Other` and `0` otherwise -- we do this so we can order by this column and put the `Other` at the bottom of our table; and, 3., it reproduces the table customer.

Then we simply calculate what we want and group the table by the country, then order by `sort` and then order by the total profit.

In [3]:
%%sql


WITH customer_grouped_one AS (
SELECT
        CASE
               WHEN (
                     SELECT count(*)
                     FROM customer
                     where country = c.country
                    ) = 1 THEN "Other"
               ELSE c.country
        END AS country,
       CASE
               WHEN (
                     SELECT count(*)
                     FROM customer
                     where country = c.country
                    ) = 1 THEN 1
               ELSE 0
        END AS sort,
        c.*
    FROM customer c
)

SELECT
    cg.country AS Country,
    COUNT(distinct cg.customer_id) AS "Total clients",
    ROUND(SUM(il.unit_price),2) AS "Total profit",
    ROUND(SUM(il.unit_price)/COUNT(distinct cg.customer_id),2) AS "Avarage of profit per customer",
    ROUND(SUM(il.unit_price)/COUNT(distinct il.invoice_id),2) AS "Avarage of profit per order"
FROM customer_grouped_one cg
INNER JOIN invoice i ON i.customer_id = cg.customer_id
INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
GROUP BY 1
ORDER BY cg.sort ASC, 3 DESC;

 * sqlite:///chinook.db
Done.


Country,Total clients,Total profit,Avarage of profit per customer,Avarage of profit per order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


There are three countries that have very few customers but when they buy, they buy big: the `Czech Republic`, `Portugal` and `India`. 